Praca Domowa nr 1, BD czwartek 8:30

Kacper Staroń

298842

Potwierdzam samodzielność przesyłanej pracy oraz niekorzystanie przeze mnie z niedozwolonych źródeł.

In [1]:
import pyodbc
from datetime import datetime

In [2]:
server = input('Nazwa serwera:')
database = input('Nazwa bazy danych:')
user = input('login:')
password = input('hasło:')

Nazwa serwera:LAPTOP-KDC4KRAQ
Nazwa bazy danych:Northwind
login:sa
hasło:sastudent


In [3]:
connection_string = 'Driver={{SQL Server}};Server={0};Database={1};Uid={2};Pwd={3}'.format(server, database, user, password) 
connection = pyodbc.connect(connection_string, autocommit = False)
cursor = connection.cursor()

a) w transakcji pierwszej usuwa istniejące rekordy z w/w tabeli badań wykonanych w ubiegłym roku

In [4]:
command = 'DELETE FROM staronk WHERE data_badania < ?'
parameters = ['20200101']

cursor.execute(command, parameters)

connection.commit()

b) w transakcji drugiej wstawia trzy rekordy w w/w tabeli na podstawie danych wprowadzonych do przez użytkownika

In [5]:
#szukamy ręcznie id dla nowych wprowadzanych rekordów
cursor.execute('select max(badanie_id) from staronk')
new_id = cursor.fetchone()[0]


command = 'insert into staronk(badanie_id, imie, nazwisko, nr_telefonu, data_badania, wynik) output INSERTED.badanie_id values (?, ?, ?, ?, ?, ?)'
for i in range(3):
    new_id += 1
    
    imie = input('Imię:')
    nazwisko = input('Nazwisko:')
    nr_telefonu = input('Numer telefonu:')
    data_badania = input('Data badania(yyyy-mm-dd):')
    wynik = input('Wynik:')

    cursor.execute(command, [new_id, imie, nazwisko, nr_telefonu, data_badania, wynik])

connection.commit()

Imię:Maciej
Nazwisko:Kot
Numer telefonu:991199119
Data badania(yyyy-mm-dd):2020-10-3
Wynik:1
Imię:Klemens
Nazwisko:Murańka
Numer telefonu:123444555
Data badania(yyyy-mm-dd):2018-3-3
Wynik:0
Imię:Kamil
Nazwisko:Stoch
Numer telefonu:987654333
Data badania(yyyy-mm-dd):2020-1-5
Wynik:0


c) w transakcji trzeciej wykonuje dwie operacje:

1. modyfikuje rekordy poprzez zmianę wyniku na pozytywny ostatniego względem daty badania

In [6]:
command = 'update staronk set wynik = 1 where data_badania = (select max(data_badania) from staronk)'
cursor.execute(command)


2. dopisuje badania dla tego samej osoby (imię, nazwisko, telefon) z datą dzisiejszą i wynikiem negatywnym

In [7]:
new_id += 1
command = 'select * from staronk where data_badania = (select max(data_badania) from staronk)'
cursor.execute(command)
rekord = cursor.fetchone()

imie = rekord[1]
nazwisko = rekord[2]
nr_telefonu = rekord[3]
data_badania = str(datetime.date(datetime.now()))
wynik = 0

command = 'insert into staronk(badanie_id, imie, nazwisko, nr_telefonu, data_badania, wynik) output INSERTED.badanie_id values (?, ?, ?, ?, ?, ?)'
cursor.execute(command, new_id, imie, nazwisko, nr_telefonu, data_badania, wynik)

connection.commit()

d) usuwa badania z datą_badania późniejszą niż data wprowadzona przez użytkownika

In [8]:
command = 'delete staronk where data_badania > ?'
parameters = input('Podaj datę(yyyy-mm-dd):')
cursor.execute(command, parameters)

connection.commit()

Podaj datę(yyyy-mm-dd):2020-3-1


e) wypisuje wszystkie rekordy w tabeli

In [9]:
command = 'select * from staronk'
cursor.execute(command)
for row in cursor:
    print(row)

connection.commit()

(4, 'Klemens', 'Murańka', '123444555', '2018-03-03', False)
(5, 'Kamil', 'Stoch', '987654333', '2020-01-05', False)


In [10]:
connection.close()